In [7]:
from typing import (Any, Callable, Dict, Generic, Iterable, List, Mapping,
                    NewType, Sequence, Tuple, TypeVar, Union)

import os
import re
import sys
import traceback

import datetime
import logging

import math

from operator import itemgetter
from subprocess import PIPE, Popen, call
import importlib

import csv
import json

import numpy as np  # type: ignore
import pandas as pd  # type: ignore

import fiona  # type: ignore
from fiona.crs import from_epsg # type: ignore
import geopandas as gpd  # type: ignore

import matplotlib.pyplot as plt  # type: ignore
import seaborn as sns  # type: ignore

from arpeggio import ParserPython, visit_parse_tree  # type: ignore

from arpeggio import RegExMatch, Optional, ZeroOrMore, OneOrMore, EOF, UnorderedGroup, And, Not, Combine  # type: ignore
from arpeggio import ParserPython, PTNodeVisitor, visit_parse_tree, NoMatch  # type: ignore

import soil_lib

from soil_lib.LoimisLookups import siffer_rules_lookup, updated_texture_error_lookup, fillers_by_numbers

from soil_lib.LoimisGrammarV2 import update_main_siffer_l, loimisp_short, loimisp_short_dk, parse_test, split_and_cut, consolidate_loimis, test_brackets, parse_reconstituate, load_default_texture_defensively, soilParts, soilParts_dk

from soil_lib.LoimisVisitor import LpVisitor, loimis_grammar_product, test_layer_depths, set_texture_values

%matplotlib inline

In [8]:
#################
# create logger
#################
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# logfile = logging.FileHandler('temp_out/soil_convert.log', "w", encoding = "utf-8")
console = logging.StreamHandler(sys.stdout)

# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(asctime)s - %(levelname)s [%(filename)s:%(lineno)d] - %(message)s')
# logfile.setFormatter(formatter)
console.setFormatter(formatter)

# add the handlers to the logger
logger.handlers = []
# logger.addHandler(logfile)
logger.addHandler(console)


In [9]:
# #############################
# global vars and path names
##############################

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (25, 10)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

soil_legend_file = "../../soil/eesti_soil/mullalegend_4.csv"
soil_legend_df = pd.read_csv(
    soil_legend_file, encoding='latin1', sep=';', quotechar='"')
soil_legend_lookup = soil_legend_df['Tähistus kaardil'].tolist()
soil_legend = set(soil_legend_lookup)

logger.debug(soil_legend_df.sample(10))

the_soilmap_dataset = '../../soil/eesti_soil/Mullakaart.shp'

soil_legend_df.sample(10)

################

siffer_update_matchinfo = 'soil_orig_siffer_update_matchinfo.csv'

parquet_checkpoint_soil_types = 'eesti_soil_work_siffers.parquet'

unique_loimis_list = 'unique_loimis1.csv'
unique_loimis_with_siffer = 'unique_loimis1_with_siffer.csv'

loimis_update_parseinfo = 'soil_orig_loimis_update_parseinfo.csv'

parquet_checkpoint_loimis_fix = 'eesti_soil_work_loimis.parquet'

parquet_checkpoint_grammar_product = 'parquet_checkpoint_grammar_product.parquet'

parquet_checkpoint_texture_values = 'parquet_checkpoint_texture_values.parquet'

shape_export_texture_values = 'eesti_soil_red1_texture_overview.shp'


## Load the original sourcefile, or a progress checkpoint or export

### soilmap shapefile from landboard

https://geoportaal.maaamet.ee/docs/muld/Mullakaart_SHP.zip?t=20170301161200

```
#> md5sum Mullakaart_SHP.zip
e6f25e2bd089926933df673798c9ac71 *Mullakaart_SHP.zip
```

In [ ]:
# pre-cleanup
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

logger.info('initialising and loading soil map: {}'.format(start))
    
eesti_soil_red1 = gpd.read_file(the_soilmap_dataset, encoding='utf-8')

eesti_soil_red1['orig_fid'] = eesti_soil_red1.index.copy()

display(eesti_soil_red1.sample(10))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

### Load Parquet checkpoint with fixed Siffers

- only a Pandas DataFrame (not Geodataframe, but has wkb_geometry column)

parquet_checkpoint_soil_types

In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

logger.info('loading soil map: {}'.format(parquet_checkpoint_soil_types))

eesti_soil_red1 = pd.read_parquet(parquet_checkpoint_soil_types)

display(eesti_soil_red1.sample(10))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

### Load Parquet checkpoint with pre-parsed and fixed loimis codes

- only a Pandas DataFrame (not Geodataframe, but has wkb_geometry column for reinit if needed)

parquet_checkpoint_loimis_fix

In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

logger.info('loading soil map: {}'.format(parquet_checkpoint_loimis_fix))

eesti_soil_red1 = pd.read_parquet(parquet_checkpoint_loimis_fix)

display(eesti_soil_red1.sample(10))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

eesti_soil_red1 = eesti_soil_red1.loc[eesti_soil_red1['Varv'].isin(
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])]


reorder = ['orig_fid', 'upd_siffer', 'WRB_code', 'Siffer', 'loimis_reconst', 'has_no_info', 'Loimis1', 'Lihtloimis', 'Huumus', 'Kivisus', 'Varv', 'Boniteet', 'Shape_Area', 'wkb_geom' ]
eesti_soil_red1 = eesti_soil_red1[reorder]

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

In [ ]:
display(eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['loimis_reconst'].isin(['no_info'])].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].sample(30))

In [ ]:
import math

def load_default_texture_defensively(row: Dict[str, Any]) -> pd.Series:

    if row['loimis_reconst'] == 'no_info' or row['Loimis1'] is None:

        try:
            found = fillers_by_numbers.get(row['upd_siffer']).get('main_texture')

            return pd.Series([found, 0])
        except KeyError as ex:
            logger.error(ex)
            return pd.Series(['no_info', 1])
        except IndexError as ex:
            logger.error(ex)
            return pd.Series(['no_info', 1])
    else:
        return pd.Series([row['loimis_reconst'], 0])


# eesti_soil_red1[['loimis_reconst', 'has_no_info']] = eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()]['upd_siffer'].apply(lambda x: load_default_texture(x))

eesti_soil_red1[['loimis_reconst', 'has_no_info']] = eesti_soil_red1.apply(load_default_texture_defensively, axis=1)

display(eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].index.size)

display(eesti_soil_red1.loc[eesti_soil_red1['loimis_reconst'].isin(['no_info'])].index.size)

display(eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].sample(30))

In [ ]:
display(eesti_soil_red1.loc[eesti_soil_red1['loimis_reconst'].isin(['no_info'])])
display(eesti_soil_red1.loc[eesti_soil_red1['has_no_info'] > 0])

### Load Parquet checkpoint with parsed loimis and grammar product

- only a Pandas DataFrame (not Geodataframe, but has wkb_geometry column)
- grammar product has to be deserialised from json

parquet_checkpoint_grammar_product

In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

logger.info('loading soil map: {}'.format(parquet_checkpoint_grammar_product))

eesti_soil_red1 = pd.read_parquet(parquet_checkpoint_grammar_product)

eesti_soil_red1['loimis_grammar'] = eesti_soil_red1['grammar_json_txt'].apply(lambda x: json.loads(x))
eesti_soil_red1.drop(['grammar_json_txt'], axis=1, inplace=True)


display(eesti_soil_red1.sample(10))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))


### Load SHP/Parquet checkpoint with grammar product, Layers, Texture values

- as SHP GeoDataFrame (or parquet with wkb_geometry column)
- grammar product has to be deserialised from json in parquet (is not existing in SHP anymore

parquet_checkpoint_texture_values = 'parquet_checkpoint_texture_values.parquet'

shape_export_texture_values = 'eesti_soil_red1_texture_overview.shp'

In [10]:
# pre-cleanup
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

logger.info('initialising and loading soil map: {}'.format(start))

eesti_soil_red1 = gpd.read_file(shape_export_texture_values, encoding='utf-8')

display(eesti_soil_red1.sample(10))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

2019-06-14 17:26:39,039 - INFO [<ipython-input-10-3fd579b02b49>:4] - initialising and loading soil map: 2019-06-14 17:26:39.039745


,orig_fid,upd_siffer,WRB_code,Boniteet,Varv,Loimis1,loimis_rec,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4,EST_TXT1,LXTYPE1,EST_TXT2,LXTYPE2,EST_TXT3,LXTYPE3,EST_TXT4,LXTYPE4,SOL_CLAY1,SOL_SILT1,SOL_SAND1,SOL_ROCK1,SOL_CLAY2,SOL_SILT2,SOL_SAND2,SOL_ROCK2,SOL_CLAY3,SOL_SILT3,SOL_SAND3,SOL_ROCK3,SOL_CLAY4,SOL_SILT4,SOL_SAND4,SOL_ROCK4,geometry
387807,387807,Kr,RG-sk.ca,39,2,r₃ls/r₄ls,r₃ls/r₄ls,1,1000,1000,0,0,0,ls,L,ls,L,None,None,None,None,15,30,55,25,15,30,55,40,0,0,0,0,0,0,0,0,"POLYGON ((599428.299999997 6558034.129999999, ..."
659176,659166,M',HS-sa,53,17,None,t₃/l,1,1000,1000,0,0,0,t3,PEAT,l,S,None,None,None,None,70,15,15,0,5,5,90,0,0,0,0,0,0,0,0,0,"POLYGON ((648452.8900000006 6413443.079999998,..."
144129,143998,GI1,GL-hi.eup,45,16,ls₂50-70/ls₃,ls₂50-70/ls₃,2,1000,600,400,0,0,ls2,L,ls3,CL,None,None,None,None,15,30,55,0,35,15,50,0,0,0,0,0,0,0,0,0,"POLYGON ((691254.2800000012 6579070.699999999,..."
541020,533289,LkIg,RT-gln.ab.dy,33,8,v⁰₁l75-100/ls,v⁰₁l75-100/ls,2,1000,875,125,0,0,l,S,ls,L,None,None,None,None,5,5,90,6,15,30,55,0,0,0,0,0,0,0,0,0,"POLYGON ((527107.3599999994 6430755.050000001,..."
702988,702590,M''',HS-sa,40,17,t₂,t₂,1,1000,1000,0,0,0,t2,PEAT,None,None,None,None,None,None,60,20,20,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((630630.2899999991 6436716.289999999,..."
732537,732525,S'',HS-hm.dy,0,11,t₁40/t₃30/pl,t₁40/t₃30/pl,3,1000,400,300,300,0,t1,PEAT,None,None,pl,S,None,None,50,25,25,0,0,0,0,0,7,3,90,0,0,0,0,0,"POLYGON ((656452.5200000033 6399208.129999999,..."
164760,164760,Go,GL-mo.can,38,14,l40-70/s,l40-70/s,2,1000,550,450,0,0,l,S,s,C,None,None,None,None,5,5,90,0,45,30,25,0,0,0,0,0,0,0,0,0,"POLYGON ((414348.3288000003 6466020.344099998,..."
604903,604757,LPe,RT-eu,40,6,s,s,1,1000,1000,0,0,0,s,C,None,None,None,None,None,None,45,30,25,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((700460.200000003 6418719.370000001, ..."
167112,167112,Go,GL-mo.can,40,14,s,s,1,1000,1000,0,0,0,s,C,None,None,None,None,None,None,45,30,25,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((489383.8800000027 6531497.98, 489382..."
90178,93358,E3o,RG-br.eu,27,18,ls,ls,1,1000,1000,0,0,0,ls,L,None,None,None,None,None,None,15,30,55,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((644639.25 6400965.809999999, 644648...."


2019-06-14 17:28:27,905 - INFO [<ipython-input-10-3fd579b02b49>:12] - elapsed seconds so far: 108


## continue from here, after loading one of the above dataframes

In [ ]:
display(eesti_soil_red1.loc[eesti_soil_red1['EST_TXT1'].isin(['no_info'])])

In [ ]:
eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].groupby('upd_siffer').count().sort_values(by='orig_fid')

In [ ]:
eesti_soil_red1.loc[eesti_soil_red1['upd_siffer'].isin(["M'''"])].groupby('loimis_rec').count().sort_values(by='orig_fid').tail(2)

In [ ]:
eesti_soil_red1.loc[eesti_soil_red1['WRB_code'].isin(["HS-sa"])].groupby('LXTYPE1').count().sort_values(by='orig_fid')

In [ ]:
eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].groupby('upd_siffer').count().sort_values(by='orig_fid').index.size

In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

eesti_soil_red1[['nlayers', 'SOL_ZMX', 'SOL_Z1', 'SOL_Z2', 'SOL_Z3', 'SOL_Z4']] = eesti_soil_red1['loimis_grammar'].apply(lambda x: test_layer_depths(x))
    
display(eesti_soil_red1.loc[eesti_soil_red1['SOL_ZMX'] < 100].sample(15))

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far (after test_layer_depths): {}'.format(int(delta.total_seconds())))

eesti_soil_red1[['EST_TXT1', 'LXTYPE1', 'SOL_CLAY1', 'SOL_SILT1', 'SOL_SAND1', 'SOL_ROCK1',
                 'EST_TXT2', 'LXTYPE2', 'SOL_CLAY2', 'SOL_SILT2', 'SOL_SAND2', 'SOL_ROCK2',
                 'EST_TXT3', 'LXTYPE3', 'SOL_CLAY3', 'SOL_SILT3', 'SOL_SAND3', 'SOL_ROCK3',
                 'EST_TXT4', 'LXTYPE4', 'SOL_CLAY4', 'SOL_SILT4', 'SOL_SAND4', 'SOL_ROCK4']] = eesti_soil_red1['loimis_grammar'].apply(lambda x: set_texture_values(x))

display(eesti_soil_red1.sample(10))


tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far (after set_texture_values): {}'.format(int(delta.total_seconds())))

display(eesti_soil_red1.loc[eesti_soil_red1['SOL_ROCK1'] > 20].count())

display(set(eesti_soil_red1['LXTYPE1'].unique()))
display(set(eesti_soil_red1['LXTYPE2'].unique()))
display(set(eesti_soil_red1['LXTYPE3'].unique()))
display(set(eesti_soil_red1['LXTYPE4'].unique()))


In [ ]:
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'].isin([''])].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'] == 0].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'].isin(['B'])].index.size)

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'].isin([''])].sample(5))
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'].isin(['B'])])
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'] == 0])

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE2'] == 0].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE2'].isin(['B'])].index.size)

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE1'].isin(['B'])])
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE2'] == 0].sample(5))

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE3'] == 0].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE3'].isin(['B'])].index.size)

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE3'].isin(['B'])])
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE3'] == 0])

display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE4'] == 0].index.size)
display(eesti_soil_red1.loc[eesti_soil_red1['LXTYPE4'].isin(['B'])].index.size)



In [ ]:
eesti_soil_red1.loc[eesti_soil_red1['upd_siffer'].isin(["M'''"])].groupby('EST_TXT1').count().sort_values(by='orig_fid').index

In [ ]:
filler = eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].groupby('upd_siffer').count().sort_values(by='orig_fid').index.tolist()

fillers_by_numbers = {}

for i in filler:
    # display(i)
    t_df = eesti_soil_red1.loc[eesti_soil_red1['upd_siffer'].isin([i])].groupby('loimis_rec').count().sort_values(by='orig_fid', ascending=False).head(2)
    # display(t_df)
    
    rec = t_df.index.tolist()
    num = t_df.orig_fid.tolist()
    # display(f"{i} {rec} {num}")
    ld = { "textures": rec, "nums": num}
    if rec[0] != "no_info":
        ld.update({"main_texture" : rec[0]})
    else:
        ld.update({"main_texture" : rec[1]})
        
    fillers_by_numbers.update({i : ld})
display(fillers_by_numbers)

In [ ]:
type(eesti_soil_red1)

In [ ]:
demo_df = eesti_soil_red1.loc[eesti_soil_red1['Loimis1'].isna()].copy()
display(demo_df.sample(10))
display(demo_df.count())

In [ ]:
def load_default_texture(siffer: str) -> str:
    try:
        return fillers_by_numbers.get(siffer).get('main_texture')
    except Exception:
        return 'no_info'

demo_df['loimis_rec'] = demo_df['upd_siffer'].apply(lambda x: load_default_texture(x))

display(demo_df.sample(10))
display(demo_df.loc[demo_df['loimis_rec'].isin(['no_info'])].count())

In [ ]:
importlib.reload(soil_lib.LoimisLookups)
importlib.reload(soil_lib.LoimisGrammarV2)
importlib.reload(soil_lib.LoimisVisitor)

from soil_lib.LoimisLookups import siffer_rules_lookup, updated_texture_error_lookup

from soil_lib.LoimisVisitor import LpVisitor, test_layer_depths, loimis_grammar_product, set_texture_values

from soil_lib.LoimisGrammarV2 import loimisp_short, loimisp_short_dk, soilParts, soilParts_dk

from soil_lib.LoimisGrammarV2 import (loimisp_short, loimisp_short_dk, split_and_cut, clean_dashes,
                                      consolidate_loimis, subscripts_consolidate, test_brackets, strip_more,
                                      bracket_matcher_inside, remove_brkt_inside, remove_inside_group, can_parse, parse_test )

t_parser = ParserPython(loimisp_short() , memoization=False)
tk_parser = ParserPython(loimisp_short_dk() , memoization=False)

sp_parser = ParserPython(soilParts , memoization=False)
sp_tk_parser = ParserPython(soilParts_dk , memoization=False)


In [ ]:
start = datetime.datetime.now() # strftime("%Y-%m-%d %H:%M:%S")

test_df[['EST_TXT1', 'LXTYPE1', 'SOL_CLAY1', 'SOL_SILT1', 'SOL_SAND1', 'SOL_ROCK1',
                 'EST_TXT2', 'LXTYPE2', 'SOL_CLAY2', 'SOL_SILT2', 'SOL_SAND2', 'SOL_ROCK2',
                 'EST_TXT3', 'LXTYPE3', 'SOL_CLAY3', 'SOL_SILT3', 'SOL_SAND3', 'SOL_ROCK3',
                 'EST_TXT4', 'LXTYPE4', 'SOL_CLAY4', 'SOL_SILT4', 'SOL_SAND4', 'SOL_ROCK4']] = test_df['loimis_grammar'].apply(lambda x: set_texture_values(x))
display(test_df.sample(15))
display(test_df.loc[test_df['SOL_Z3'] > 3])

tmp_step = datetime.datetime.now()
delta = tmp_step - start
logger.info('elapsed seconds so far: {}'.format(int(delta.total_seconds())))

display(test_df.sample(15))


In [ ]:
checks = fix_depths

test_df = pd.DataFrame({'Loimis1': checks})

test_df['split_layered'] = test_df['Loimis1'].astype(object).fillna("no_info").apply(lambda x: split_and_cut(x))
test_df['num_brackets_fixed'] = test_df['split_layered'].apply(lambda x: test_brackets(x))
test_df[['test_parse','test_parse_errors']] = test_df['num_brackets_fixed'].apply(lambda x: parse_test(x, t_parser, tk_parser))
test_df[['loimis_reconst','has_no_info']] = test_df['test_parse'].apply(lambda x: parse_reconstituate(x))
test_df[['loimis_grammar','parse_info']] = test_df['loimis_reconst'].apply(lambda x: loimis_grammar_product(x, sp_tk_parser))
test_df[['nlayers', 'SOL_ZMX', 'SOL_Z1', 'SOL_Z2', 'SOL_Z3', 'SOL_Z4']] = test_df['loimis_grammar'].apply(lambda x: test_layer_depths(x))


display(test_df.loc[test_df['SOL_ZMX'] > 3000])
display(test_df.loc[test_df['SOL_ZMX'] > 3000]['Loimis1'].unique())
display(test_df.loc[test_df['SOL_ZMX'] > 3000]['parse_info'].unique())
display(test_df.loc[test_df['SOL_ZMX'] < 100].count())
display(test_df.loc[test_df['SOL_ZMX'] > 3000].count())

display(test_df.sample(15))
display(test_df.loc[test_df['SOL_Z4'] > 1])

for e in test_df['loimis_grammar'].iteritems():
    for s in e[1]['soilparts']:
        has_kr = False
        
        for c in s['constituents']:
            if c['type'] == 'kores':
                try:
                    kores_type = c['code']
                except IndexError:
                    has_kr = True
                    pass
                except KeyError:
                    has_kr = True
                    pass
        if has_kr:
            display(e[1])
        

In [ ]:
visit_parse_tree(parse_tree=sp_tk_parser.parse('plsl35-45/ls60'), visitor=LpVisitor(debug=True))

In [ ]:
# 'pk', 'kr', 'p', 'd', 'lu', 'ck'
visit_parse_tree(parse_tree=sp_tk_parser.parse('v⁰₁ls₁50-75/v⁰₂ls₂/kr'), visitor=LpVisitor(debug=True))